In [ ]:
import numpy as np
import pandas as pd

In [ ]:
test_df=pd.read_csv('/content/test.csv',header=0)

In [ ]:
num_img=test_df.values.astype(float)
num_img/=255
num_img.shape
num_img=num_img.reshape(num_img.shape[0],28,28,1)

In [2]:
from matplotlib import pyplot as plt

In [ ]:
train_df=pd.read_csv('/content/train.csv')

In [ ]:
from sklearn.model_selection import train_test_split
train_data,testdata=train_test_split(train_df,test_size=0.1)
x_train=train_data.iloc[:,1:]
x_test=testdata.iloc[:,1:]
y_train=train_data.iloc[:,0]
y_test=testdata.iloc[:,0]

In [ ]:
y_train.shape

In [ ]:
x_train.shape

In [ ]:
y_test.shape

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten,Conv2D,MaxPooling2D
from keras.optimizers import RMSprop

In [ ]:
img_rows,img_cols=28,28

In [ ]:
x_train=x_train.values
x_test=x_test.values

In [ ]:
x_train.shape

In [ ]:
x_train=x_train.reshape(x_train.shape[0],img_rows,img_cols,1).astype(float)
x_test =x_test.reshape(x_test.shape[0],img_rows,img_cols,1).astype(float)

x_train/=255
x_test/=255


In [ ]:
num_classes=10

In [ ]:
y_train=keras.utils.to_categorical(y_train,num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)

In [ ]:
y_test.shape

In [ ]:
print(x_train.shape)
model=Sequential()
model.add(Conv2D(32,(5,5),activation='relu',padding='same',input_shape=(img_rows,img_cols,1)))
model.add(Conv2D(32,(5,5),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
model.add(Conv2D(64,(3,3),activation='relu',padding='same'))
model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
model.add(Conv2D(128,(3,3),activation='relu',padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes,activation='softmax'))
optimizer=RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=optimizer,
              metrics=['accuracy'])
hist=model.fit(x_train,y_train,batch_size=32,epochs=5,validation_data=(x_test,y_test))

In [ ]:

score=model.evaluate(x_test,y_test)

epochs=range(1,len(hist.history['accuracy'])+1)
plt.plot(epochs, hist.history['loss'], label='Training loss', ls='-') #損失値
plt.plot(epochs, hist.history['val_loss'], label='Validation loss')
plt.title('Training and Validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

plt.plot(epochs, hist.history['accuracy'],  label='Training acc') #正解率
plt.plot(epochs, hist.history['val_accuracy'], label="Validation acc")
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
model.save('./cnn.num')

In [ ]:
ans=model.predict(num_img)
ans.shape

In [ ]:
results=np.argmax(ans,axis=1)

In [ ]:
results=pd.Series(results,name='Label')

In [ ]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

In [ ]:
submission.to_csv("cnn_mnist_datagen.csv",index=False)